## Imports

In [9]:
import os

from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import numpy as np

import matplotlib.pyplot as plt

from knoten.bundle import *

## Load Network and Generate Sensors

In [10]:
cubes = '/scratch/csm2020/data/cubes2.lis'
sensors = generate_sensors(cubes)

network = '/scratch/csm2020/data/hand_dense.net'
cnet = io_controlnetwork.from_isis(network)
cnet = compute_apriori_ground_points(cnet, sensors) # autoseed did not generate ground points, calculate and repopulate the data frame

## Determine Which Sensor Parameters to Solve For

In [11]:
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()}
for sn, parameters in all_parameters.items():
    print(f"Image: {sn}")
    for param in parameters:
        print(f"  {param.name} | {param.index} | {param.value}")

1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
Image: MRO/CTX/1085197697:073
  IT Pos. Bias    | 0 | 0.0
  CT Pos. Bias    | 1 | 0.0
  Rad Pos. Bias   | 2 | 0.0
  IT Vel. Bias    | 3 | 0.0
  CT Vel. Bias    | 4 | 0.0
  Rad Vel. Bias   | 5 | 0.0
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omega Rate      | 9 | 0.0
  Phi Rate        | 10 | 0.0
  Kappa Rate      | 11 | 0.0
  Omega Accl      | 12 | 0.0
  Phi Accl        | 13 | 0.0
  Kappa Accl      | 14 | 0.0
  Focal Bias      | 15 | 0.0
Image: MRO/CTX/1096561308:045
  IT Pos. Bias    | 0 | 0.0
  CT Pos. Bias    | 1 | 0.0
  Rad Pos. Bias   | 2 | 0.0
  IT Vel. Bias    | 3 | 0.0
  CT Vel. Bias    | 4 | 0.0
  Rad Vel. Bias   | 5 | 0.0
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omeg

## Solve for angles and angular rates

In [12]:
solve_parameters = {sn: params[6:12] for sn, params in all_parameters.items()}

## Compute the Column Indices for Parameters

In [13]:
column_dict = compute_coefficient_columns(cnet, sensors, solve_parameters)

OrderedDict([('MRO/CTX/1085197697:073', (0, 6)), ('MRO/CTX/1157902986:250', (6, 12)), ('MRO/CTX/1096561308:045', (12, 18)), ('MRO/CTX/1136952576:186', (18, 24)), ('autoseed_001', (24, 27)), ('autoseed_002', (27, 30)), ('autoseed_003', (30, 33)), ('autoseed_004', (33, 36)), ('autoseed_005', (36, 39)), ('autoseed_006', (39, 42)), ('autoseed_007', (42, 45)), ('autoseed_008', (45, 48)), ('autoseed_009', (48, 51)), ('autoseed_010', (51, 54)), ('autoseed_011', (54, 57)), ('autoseed_012', (57, 60)), ('autoseed_013', (60, 63)), ('autoseed_014', (63, 66)), ('autoseed_015', (66, 69)), ('autoseed_016', (69, 72)), ('autoseed_017', (72, 75)), ('autoseed_018', (75, 78)), ('autoseed_019', (78, 81)), ('hand_01', (81, 84)), ('hand_02', (84, 87)), ('hand_03', (87, 90)), ('hand_04', (90, 93)), ('hand_05', (93, 96)), ('hand_06', (96, 99)), ('hand_07', (99, 102)), ('hand_08', (102, 105)), ('hand_09', (105, 108)), ('hand_10', (108, 111)), ('hand_11', (111, 114)), ('hand_12', (114, 117)), ('hand_13', (117, 1

## Compute the Weight Matrix
#### According to the weighted Normal equation (J.TWJ), W needs to be a square matrix the size of (# of measures)x2. So it is the weight of the observations. In ISIS, the weight of the observations are an inverted function of the size of the pixels on the focal plane (resolution). However, in csm we do not have access to that information. 
#### For the time being, since we are working exclusively with CTX images we are going to set the weight matrix equal to the identity matrix -> all observations have the same weight.

In [14]:
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations) # this is a place holder until Jesse adds his calculations
W_params = compute_parameter_weights(cnet, sensors, solve_parameters, column_dict)

## Calculate Initial Sigma0

In [15]:
def compute_sigma(V, dX, W_parameters, W_observations):
    """
    Computes the resulting standard deviation of the residuals for the current state of the bundle network.
    
    Parameters
    ----------
    V  :  np.array
          An array of residuals of the difference between registered measure 
          and back projected ground points in image space.
    W_parameters  :  ndarray 
                     The parameter weight matrix (i.e.: sensor parameters and point weights)
    W_observations  :  ndarray
                     The observation weight matrix (i.e.: point weights)
    
    Returns
    -------
       : float64
         Standard deviation of the residuals
    """
    num_parameters = W_parameters.shape[0]
    num_observations = W_observations.shape[0]
    dof = num_observations - num_parameters
    VTPV = V.dot(W_observations).dot(V) + dX.dot(W_parameters).dot(dX)
    sigma0 = np.sqrt(VTPV/dof)
    return sigma0

## Update Sensors and Ground Points

In [16]:
def update_parameters(sensors, parameters, network, updates, coefficient_columns):
    """
    Updates the sensor objects parameter values and the ground point values in the 
    networks DataFrame. The update occurs directly to variables, so nothing is returned.
    
    Parameters
    ----------
    sensors  :  dict
                A dictionary that maps ISIS serial numbers to CSM sensors
    parameters :  list
                  The list of  CsmParameter to compute the partials W.R.T.
    network  :  DataFrame
                The control network as a dataframe generated by plio.
    updates   : np.ndarray
                An array of updated parameter values
    coefficient_columns:  OrderedDict
                          Dictionary that maps serial numbers and point IDs to
                          the column range their parameters are in the Jacobian
                          matrix.
    
    Returns
    -------
    
    """
    # update the sensor partials
    for sn, sensor in sensors.items():
        for i, param in enumerate(parameters[sn]):
            if i > coefficient_columns[sn][1]:
                print('THIS SHOULD BE AN ACTUAL ERROR')
            current_value = sensor.getParameterValue(param.index)
            sensor.setParameterValue(param.index, current_value+updates[coefficient_columns[sn][0]+i])

    # update ground points
    for _, row in network.iterrows():
        point_id = row['id']
        ground_pt = row[['adjustedX', 'adjustedY', 'adjustedZ']].values
        adj = updates[coefficient_columns[point_id][0]:coefficient_columns[point_id][1]] 
        network.loc[network.id == point_id, ["adjustedX", "adjustedY", "adjustedZ"]] = ground_pt + adj

## Whole bundle process in a loop without LM terms

In [28]:
## sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

### INPUTS ###
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity
##############

column_dict = compute_coefficient_columns(cnet, sensors, parameters)
num_parameters = max(col_range[1] for col_range in column_dict.values())
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations)
W_params = compute_parameter_weights(cnet, sensors, parameters, column_dict)

iteration = 0
V = compute_residuals(cnet, sensors)
dX = np.zeros(W_params.shape[0]) #initialize for sigma calculatio
sigma0 = compute_sigma(V, dX, W_params, W_observations)
print(f'iteration {iteration}: sigma0 = {sigma0}\n')

max_iterations = 100
tol = 1e-10
total_correction = np.zeros(num_parameters)
damping = 0.1
for i in range(max_iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J = compute_jacobian(cnet, sensors, parameters, column_dict)    
    N = J.T.dot(W_observations).dot(J) + W_params # calculate the normal equation
    C = J.T.dot(W_observations).dot(V) - W_params.dot(total_correction)
    dX = np.linalg.inv(N).dot(C) # calculate change in camera parameters and ground points
    

    total_correction += dX
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    update_parameters(sensors, parameters, cnet, dX, column_dict)
    
    V = compute_residuals(cnet, sensors)
    sigma0 = compute_sigma(V, dX, W_params, W_observations)
#     sigma0 = np.sqrt((V.dot(W_observations).dot(V) + dX.dot(W_params).dot(dX))/dof)
    if (sigma0 < old_sigma0): #good, do we really want to make the step size smaller? 
        damping*=0.1
    else:
        damping*=10
        
    print(f'iteration {iteration}: sigma0 = {sigma0}\n')
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break
    

1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
OrderedDict([('MRO/CTX/1085197697:073', (0, 6)), ('MRO/CTX/1157902986:250', (6, 12)), ('MRO/CTX/1096561308:045', (12, 18)), ('MRO/CTX/1136952576:186', (18, 24)), ('autoseed_001', (24, 27)), ('autoseed_002', (27, 30)), ('autoseed_003', (30, 33)), ('autoseed_004', (33, 36)), ('autoseed_005', (36, 39)), ('autoseed_006', (39, 42)), ('autoseed_007', (42, 45)), ('autoseed_008', (45, 48)), ('autoseed_009', (48, 51)), ('autoseed_010', (51, 54)), ('autoseed_011', (54, 57)), ('autoseed_012', (57, 60)), ('autoseed_013', (60, 63)), ('autoseed_014', (63, 66)), ('autoseed_015', (66, 69)), ('autoseed_016', (69, 72)), ('autoseed_017', (72, 75)), ('autoseed_018', (75, 78)), ('autoseed_019', (78, 81)), ('hand_01', (81, 84)), ('hand_02', (84, 87)), ('hand_03', (87, 90)), ('hand_04

## Whole (Dense) Bundle Process in a loop with LM terms

The next three cells show examples of the bundle adjustment loop running with Levenberg-Marquat added.

This example shows the first heuristic for initializing $\lambda$, the weighting parameter, from Numerical Recipes in C++

NumericalRecipes in C++ recommended starting with $\lambda$ = 0.01, and then increasing or decreasing this value by an order of magnitude each iteration depending on whether the last iteration increased or decreased 
$\sigma_0$.

Step sizes will get smaller as the target is approached.

### LM using Numerical Recipies's heuristic

$\lambda = 0.01$ to start out, then increased or decreased by an order of magnitude each step. 

`damping` is used as the variable for $\lambda$, since in python `lambda` is a reserved word.

In [30]:
## sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

### INPUTS ###
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity
##############

column_dict = compute_coefficient_columns(cnet, sensors, parameters)
num_parameters = max(col_range[1] for col_range in column_dict.values())
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations)
W_params = compute_parameter_weights(cnet, sensors, parameters, column_dict)

iteration = 0
V = compute_residuals(cnet, sensors)
dX = np.zeros(W_params.shape[0]) #initialize for sigma calculatio
sigma0 = compute_sigma(V, dX, W_params, W_observations)
print(f'iteration {iteration}: sigma0 = {sigma0}\n')

max_iterations = 100
tol = 1e-10
total_correction = np.zeros(num_parameters)
damping = 100
for i in range(max_iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J = compute_jacobian(cnet, sensors, parameters, column_dict)    
    C = J.T.dot(W_observations).dot(V) - damping*np.identity(W_params.shape[0]).dot(total_correction) - W_params.dot(total_correction)
    N = J.T.dot(W_observations).dot(J) + damping*np.identity(W_params.shape[0]) + W_params
    dX = np.linalg.inv(N).dot(C) # calculate change in camera parameters and ground points
    
    total_correction += dX
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    update_parameters(sensors, parameters, cnet, dX, column_dict)
    
    V = compute_residuals(cnet, sensors)
    sigma0 = compute_sigma(V, dX, W_params, W_observations)
    if (sigma0 < old_sigma0):
        damping*=0.1
    else:
        damping*=10
        
    print(f'iteration {iteration}: sigma0 = {sigma0}\n')
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break
    

1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
OrderedDict([('MRO/CTX/1085197697:073', (0, 6)), ('MRO/CTX/1157902986:250', (6, 12)), ('MRO/CTX/1096561308:045', (12, 18)), ('MRO/CTX/1136952576:186', (18, 24)), ('autoseed_001', (24, 27)), ('autoseed_002', (27, 30)), ('autoseed_003', (30, 33)), ('autoseed_004', (33, 36)), ('autoseed_005', (36, 39)), ('autoseed_006', (39, 42)), ('autoseed_007', (42, 45)), ('autoseed_008', (45, 48)), ('autoseed_009', (48, 51)), ('autoseed_010', (51, 54)), ('autoseed_011', (54, 57)), ('autoseed_012', (57, 60)), ('autoseed_013', (60, 63)), ('autoseed_014', (63, 66)), ('autoseed_015', (66, 69)), ('autoseed_016', (69, 72)), ('autoseed_017', (72, 75)), ('autoseed_018', (75, 78)), ('autoseed_019', (78, 81)), ('hand_01', (81, 84)), ('hand_02', (84, 87)), ('hand_03', (87, 90)), ('hand_04

### sba/levmar's heuristic

$\lambda$ initially set to a user supplied value (or default = 0.001) * the max value in $J^{T}J$

Updates either:

(1) $\lambda = max(1/3, 1-(2\rho -1)^3) , \nu = 2.0$

(2) $\lambda = \lambda\nu, \nu = 2.0\nu$

See: http://users.ics.forth.gr/~argyros/mypapers/2004_08_tr340_forth_sba.pdf

In [37]:
## sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

### INPUTS ###
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity
##############

column_dict = compute_coefficient_columns(cnet, sensors, parameters)
num_parameters = max(col_range[1] for col_range in column_dict.values())
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations)
W_params = compute_parameter_weights(cnet, sensors, parameters, column_dict)

iteration = 0
V = compute_residuals(cnet, sensors)
dX = np.zeros(W_params.shape[0]) #initialize for sigma calculatio
sigma0 = compute_sigma(V, dX, W_params, W_observations)
print(f'iteration {iteration}: sigma0 = {sigma0}\n')

max_iterations = 100
tol = 1e-10
total_correction = np.zeros(num_parameters)
damping = 0.001
for i in range(max_iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J = compute_jacobian(cnet, sensors, parameters, column_dict)
    if i==0:
        damping *= np.max(J.T.dot(J))
    
    C = J.T.dot(W_observations).dot(V) - damping*np.identity(W_params.shape[0]).dot(total_correction) - W_params.dot(total_correction)
    N = J.T.dot(W_observations).dot(J) + damping*np.identity(W_params.shape[0]) + W_params
#    N = J.T.dot(W_observations).dot(J) + damping*W_params
#    C = J.T.dot(W_observations).dot(V) - damping*W_params.dot(total_correction) #different :D
    dX = np.linalg.inv(N).dot(C) # calculate change in camera parameters and ground points
    

    total_correction += dX
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    update_parameters(sensors, parameters, cnet, dX, column_dict)
    
    old_V = V
    V = compute_residuals(cnet, sensors)
    sigma0 = compute_sigma(V, dX, W_params, W_observations)
    if (sigma0 < old_sigma0):
        damping = max(1.0/3.0, (np.linalg.norm(old_V)**2 - np.linalg.norm(V)**2)/(dX.T.dot(damping*dX + C)))
        nu = 2
        print("DAMPING1:", damping, "NU1:", nu)
    else:
        damping = damping * nu
        nu = 2 * nu
        print("DAMPING2:", damping, "NU2:", nu)

    print(f'iteration {iteration}: sigma0 = {sigma0}\n')
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break
    

1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
1
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
OrderedDict([('MRO/CTX/1085197697:073', (0, 6)), ('MRO/CTX/1157902986:250', (6, 12)), ('MRO/CTX/1096561308:045', (12, 18)), ('MRO/CTX/1136952576:186', (18, 24)), ('autoseed_001', (24, 27)), ('autoseed_002', (27, 30)), ('autoseed_003', (30, 33)), ('autoseed_004', (33, 36)), ('autoseed_005', (36, 39)), ('autoseed_006', (39, 42)), ('autoseed_007', (42, 45)), ('autoseed_008', (45, 48)), ('autoseed_009', (48, 51)), ('autoseed_010', (51, 54)), ('autoseed_011', (54, 57)), ('autoseed_012', (57, 60)), ('autoseed_013', (60, 63)), ('autoseed_014', (63, 66)), ('autoseed_015', (66, 69)), ('autoseed_016', (69, 72)), ('autoseed_017', (72, 75)), ('autoseed_018', (75, 78)), ('autoseed_019', (78, 81)), ('hand_01', (81, 84)), ('hand_02', (84, 87)), ('hand_03', (87, 90)), ('hand_04